In [2]:
import pandas as pd

In [3]:
with open('sample_files/ISL69269-0 0x60f.txt') as file:
    file1_contents = file.read()

In [4]:
with open('sample_files/ISL69269-0 0x60g.txt') as file:
    file2_contents = file.read()

In [5]:
lines1        = file1_contents.split('\n')
lines2        = file2_contents.split('\n')

commands1     = lines1[9:-1]
commands2     = lines2[9:-1]

In [51]:
class Config_cmdline:
    def __init__(self,line):
        self.line = line
        self.name = self.line[:20].rstrip()
        self.value = self.line[37:47].rstrip()
        self.addr  = self.line[61:].rstrip()
        
        self.add_leading_0byte_if_r()  
        
    def add_leading_0byte_if_r(self):
        if self.name[0] == 'r' and len(self.addr)==1:
            self.name.append('00') #is trailing since has been reversed by data_to_list()
        
def create_command_dict(commands):
    name   = [Config_cmdline(line).name  for line in commands]
    value  = [Config_cmdline(line).value for line in commands]
    addr   = [Config_cmdline(line).addr  for line in commands]
    return {'name':name, 'value': value, 'address': addr}
    
        

In [56]:
command1_dict = create_command_dict(commands1)
command2_dict = create_command_dict(commands2)

In [57]:
df1 = pd.DataFrame(command1_dict)
df2 = pd.DataFrame(command2_dict)

In [62]:
df_diff1=df1.merge(df2, how = 'outer',indicator=True).loc[lambda x : x['_merge']=='left_only']
df_diff1

,name,value,address,_merge
19456,rGUI5,A1A08E17,3F,left_only
58274,oocLimit,05DC,46,left_only
58524,slowOcLimit,05DC,EA,left_only


In [63]:
df_diff2=df1.merge(df2, how = 'outer',indicator=True).loc[lambda x : x['_merge']=='right_only']
df_diff2

,name,value,address,_merge
58798,rGUI5,E4C06063,3F,right_only
58799,oocLimit,03E8,46,right_only
58800,slowOcLimit,04E2,EA,right_only


In [64]:
df2['name'].equals(df1['name'])

True

In [65]:
df2['address'].equals(df1['address'])

True

In [66]:
pd.concat([df1,df2]).drop_duplicates(keep=False)


,name,value,address
136,rGUI5,A1A08E17,3F
289,oocLimit,05DC,46
481,slowOcLimit,05DC,EA
136,rGUI5,E4C06063,3F
289,oocLimit,03E8,46
481,slowOcLimit,04E2,EA


In [67]:
set(df1.value).symmetric_difference(df2.value)

{'04E2', '05DC', 'A1A08E17', 'E4C06063'}